In [15]:
import tensorflow as tf
import numpy as np

In [6]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

In [47]:
#setting varialbles to be changed easily
inp_width=28
inp_height=28
channels=1
inp_pixels=784
#units in convultional layer
n_conv1=32
n_conv2=64
#filter size
conv1_k=5
conv2_k=5
stride1=1
stride2=1
#pool size
max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_output=10

inp_size_to_hidden=(inp_width//(max_pool1_k*max_pool2_k))*(inp_height//(max_pool1_k*max_pool2_k))*n_conv2

In [8]:
#weights and baises
weights={
    "wc1":tf.Variable(tf.random_normal_initializer()(shape=[conv1_k,conv1_k,channels,n_conv1])),
    "wc2":tf.Variable(tf.random_normal_initializer()(shape=[conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1":tf.Variable(tf.random_normal_initializer()(shape=[inp_size_to_hidden,n_hidden])),
    "wo":tf.Variable(tf.random_normal_initializer()(shape=[n_hidden,n_output]))
}

biases={
    "bc1":tf.Variable(tf.random_normal_initializer()(shape=[n_conv1])),
    "bc2":tf.Variable(tf.random_normal_initializer()(shape=[n_conv2])),
    "bh1":tf.Variable(tf.random_normal_initializer()(shape=[n_hidden])),
    "bo":tf.Variable(tf.random_normal_initializer()(shape=[n_output]))
}

In [9]:
def conv(x,weight,bias,stride=1):
    #apply the filter on images
    out=tf.nn.conv2d(x,weight,padding="SAME",strides=[1,stride,stride,1])
    out=tf.nn.bias_add(out,bias)
    out=tf.nn.relu(out)
    return out
def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding='SAME',ksize=[1,k,k,1],strides=[1,k,k,1])

In [10]:
def cnn(x,weights,biases):
    #first reshape the input
    x=tf.reshape(x,shape=[-1,inp_height,inp_width,channels])
    conv1=conv(x,weights['wc1'],biases['bc1'],stride1)
    conv1_pool=maxpooling(conv1,max_pool1_k)
    conv2=conv(conv1_pool,weights['wc2'],biases['bc2'],stride2)
    conv2_pool=maxpooling(conv2,max_pool2_k)
    hidden_inp=tf.reshape(conv2_pool,shape=[-1,inp_size_to_hidden])
    hidden_out_before_activation=tf.add(tf.matmul(hidden_inp,weights['wh1']),biases['bh1'])
    hidden_output=tf.nn.relu(hidden_out_before_activation)
    
    output=tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output

In [13]:

#Using Tensorflow-:
# x = tf.keras.Input(shape=(inp_pixels,), dtype=tf.float32)
# y = tf.keras.Input(shape=(n_output,), dtype=tf.int32)
# input_shape = (None, inp_pixels)

# Create input placeholders using tf.compat.v1.placeholder
# x = tf.compat.v1.placeholder(tf.float32, shape=input_shape, name='input_x')
# y = tf.compat.v1.placeholder(tf.float32, shape=(None, n_output), name='input_y')
# pred=cnn(x,weights,biases)

In [29]:
# cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))

In [16]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
# optimize = optimizer.minimize(cost)

**Using Keras to Implement CNN**

In [48]:
input_layer = tf.keras.layers.Input(shape=(inp_height, inp_width, channels))
conv_layer1=tf.keras.layers.Conv2D(filters=n_conv1,kernel_size=(conv1_k,conv1_k),strides=(stride1,stride1),padding="SAME", activation="relu")(input_layer)
pool_layer_1 = tf.keras.layers.MaxPooling2D(pool_size=(max_pool1_k,max_pool1_k), strides=(2,2),padding="VALID")(conv_layer1)
conv_layer2=tf.keras.layers.Conv2D(filters=n_conv2,kernel_size=(conv2_k,conv2_k),strides=(stride2,stride2),padding="SAME", activation="relu")(pool_layer_1)
pool_layer_2= tf.keras.layers.MaxPooling2D(pool_size=(max_pool2_k,max_pool2_k), strides=(2,2), padding="VALID")(conv_layer2)
flatten_layer = tf.keras.layers.Flatten()(pool_layer_2)
hidden_layer = tf.keras.layers.Dense(units=1024, activation="relu")(flatten_layer)
output_layer=tf.keras.layers.Dense(units=10,activation="softmax")(hidden_layer)

In [49]:
model = tf.keras.models.Model(inputs=input_layer, outputs=hidden_layer)

# Compile the model and specify loss and optimizer
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_19 (Conv2D)          (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 14, 14, 64)        51264     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 7, 7, 64)         0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 3136)              0   

Total params: 3,264,384
Trainable params: 3,264,384
Non-trainable params: 0
_________________________________________________________________
